# Script to download NOAA wave model data

**Date:** 25-01-2019

**Purpose:** Downloads NOAA model data for wave buoy locations

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
import io
from multiprocessing.pool import ThreadPool as Pool
import xarray as xr
import fetch_online_des_waves

Generate a list of model run urls

In [2]:
def build_urls():
    date = dt.datetime.strftime(dt.datetime.utcnow() - dt.timedelta(days=1), "%Y%m%d")
    baseurl = 'http://nomads.ncep.noaa.gov:9090/dods/wave/mww3/'
    runs = ['_00z','_06z','_12z','_18z']
    name = """/multi_1.glo_30mext"""
    
    urls = []
    for r in runs:
        urls.append(baseurl+date+name+date+r)
    return urls


['http://nomads.ncep.noaa.gov:9090/dods/wave/mww3/20190124/multi_1.glo_30mext20190124_00z',
 'http://nomads.ncep.noaa.gov:9090/dods/wave/mww3/20190124/multi_1.glo_30mext20190124_06z',
 'http://nomads.ncep.noaa.gov:9090/dods/wave/mww3/20190124/multi_1.glo_30mext20190124_12z',
 'http://nomads.ncep.noaa.gov:9090/dods/wave/mww3/20190124/multi_1.glo_30mext20190124_18z']

In [6]:
def get_location():
    datadf = fetch_online_des_waves.get_waves()
    sites = list(datadf.Site.unique())
    lats = []
    longs = []
    site = []
    for i in sites:
        if str(datadf['Longitude'][datadf['Site'] == i][-1]) == '-99.9':
            continue
        if str(datadf['Latitude'][datadf['Site'] == i][-1]) == '-99.9':
            continue
        longs.append(str(datadf['Longitude'][datadf['Site'] == i][-1]))
        lats.append(str(datadf['Latitude'][datadf['Site'] == i][-1]))
        site.append(i)
    coords = list(zip(lats, longs, site))
    return coords


[('-28.17923', '153.57743', 'Tweed Heads'),
 ('-26.84698', '153.15566', 'Caloundra'),
 ('-27.96552', '153.44253', 'Gold Coast')]

In [9]:
def get_data(url):
    """takes in a url and returns the data from the NOAA Multgrid wave model for each wave buoy location
    """
    dataset = xr.open_dataset(url)
    locs = get_location()
    dataframes = []
    for loc in locs:
        sitedata = dataset.sel(lon=float(loc[1]), lat=float(loc[0]), method='nearest')
        df = sitedata.htsgwsfc.to_dataframe()
        df['perpwsfc'] = sitedata.perpwsfc.data
        df['dirpwsfc'] = sitedata.dirpwsfc.data
        df['site'] = loc[-1]
        dataframes.append(df)
    data = pd.concat(dataframes)
    return data


lat    lon  htsgwsfc  perpwsfc   dirpwsfc         site
time                                                                        
2019-01-24 00:00:00 -28.0  153.5      0.79  8.090000  83.339996  Tweed Heads
2019-01-24 03:00:00 -28.0  153.5      0.75  8.020000  83.909996  Tweed Heads
2019-01-24 06:00:00 -28.0  153.5      0.74  7.970000  84.369995  Tweed Heads
2019-01-24 09:00:00 -28.0  153.5      0.73  7.920000  84.720001  Tweed Heads
2019-01-24 12:00:00 -28.0  153.5      0.70  7.870000  84.839996  Tweed Heads
2019-01-24 15:00:00 -28.0  153.5      0.67  7.810000  84.779999  Tweed Heads
2019-01-24 18:00:00 -28.0  153.5      0.65  7.750000  84.669998  Tweed Heads
2019-01-24 21:00:00 -28.0  153.5      0.63  7.680000  84.659996  Tweed Heads
2019-01-25 00:00:00 -28.0  153.5      0.61  7.580000  80.970001  Tweed Heads
2019-01-25 03:00:00 -28.0  153.5      0.64  7.470000  81.290001  Tweed Heads
2019-01-25 06:00:00 -28.0  153.5      0.67  7.390000  81.709999  Tweed Heads
2019-01-25 09:00:00 -28.0  153.5      0.67  7.310000  82.169998  Tweed Heads
2019-01-25 12:00:00 -28.0  153.5      0.70  7.250000  82.400002  Tweed Heads
2019-01-25 15:00:00 -28.0  153.5      0.75  7.200000  82.709999  Tweed Heads
2019-01-25 18:00:00 -28.0  153.5      0.77  7.000000  78.099998  Tweed Heads
2019-01-25 21:00:00 -28.0  153.5      0.78  5.810000  67.489998  Tweed Heads
2019-01-26 00:00:00 -28.0  153.5      0.79  5.880000  70.010002  Tweed Heads
2019-01-26 03:00:00 -28.0  153.5      0.85  5.970000  69.979996  Tweed Heads
2019-01-26 06:00:00 -28.0  153.5      0.90  6.090000  69.580002  Tweed Heads
2019-01-26 09:00:00 -28.0  153.5      0.92  6.270000  70.799995  Tweed Heads
2019-01-26 12:00:00 -28.0  153.5      0.94  6.410000  69.629997  Tweed Heads
2019-01-26 15:00:00 -28.0  153.5      0.97  6.470000  68.939995  Tweed Heads
2019-01-26 18:00:00 -28.0  153.5      0.99  6.540000  68.709999  Tweed Heads
2019-01-26 21:00:00 -28.0  153.5      1.02  6.690000  70.809998  Tweed Heads
2019-01-27 00:00:00 -28.0  153.5      1.04  6.820000  70.449997  Tweed Heads
2019-01-27 03:00:00 -28.0  153.5      1.07  6.930000  70.059998  Tweed Heads
2019-01-27 06:00:00 -28.0  153.5      1.07  6.990000  69.949997  Tweed Heads
2019-01-27 09:00:00 -28.0  153.5      1.09  7.030000  70.019997  Tweed Heads
2019-01-27 12:00:00 -28.0  153.5      1.13  7.050000  69.849998  Tweed Heads
2019-01-27 15:00:00 -28.0  153.5      1.15  7.060000  69.479996  Tweed Heads
...                   ...    ...       ...       ...        ...          ...
2019-01-27 21:00:00 -28.0  153.5      1.08  7.090000  68.970001   Gold Coast
2019-01-28 00:00:00 -28.0  153.5      1.03  7.070000  68.729996   Gold Coast
2019-01-28 03:00:00 -28.0  153.5      1.00  7.020000  68.519997   Gold Coast
2019-01-28 06:00:00 -28.0  153.5      0.99  6.960000  68.449997   Gold Coast
2019-01-28 09:00:00 -28.0  153.5      0.97  6.900000  68.650002   Gold Coast
2019-01-28 12:00:00 -28.0  153.5      0.99  6.840000  69.379997   Gold Coast
2019-01-28 15:00:00 -28.0  153.5      1.01  6.730000  70.529999   Gold Coast
2019-01-28 18:00:00 -28.0  153.5      1.01  6.570000  67.659996   Gold Coast
2019-01-28 21:00:00 -28.0  153.5      0.99  6.520000  68.589996   Gold Coast
2019-01-29 00:00:00 -28.0  153.5      0.96  6.490000  69.279999   Gold Coast
2019-01-29 03:00:00 -28.0  153.5      0.95  6.450000  69.879997   Gold Coast
2019-01-29 06:00:00 -28.0  153.5      0.94  6.390000  70.559998   Gold Coast
2019-01-29 09:00:00 -28.0  153.5      0.93  9.719999  95.229996   Gold Coast
2019-01-29 12:00:00 -28.0  153.5      0.97  9.650000  95.329994   Gold Coast
2019-01-29 15:00:00 -28.0  153.5      1.03  6.080000  73.049995   Gold Coast
2019-01-29 18:00:00 -28.0  153.5      1.09  6.040000  73.729996   Gold Coast
2019-01-29 21:00:00 -28.0  153.5      1.14  6.340000  76.000000   Gold Coast
2019-01-30 00:00:00 -28.0  153.5      1.18  6.820000  80.110001   Gold Coast
2019-01-30 03:00:00 -28.0  153.5      1.22  7.170000  83.419998   Gold Coast
2019-01-30 06:00:0

In [1]:
def main():
    pool = Pool(4) # or whatever your hardware can support
    urls = build_urls()
    dfs = pool.map(get_data,  urls)
    data = pd.concat(dfs)
    return data